# Carga del dataset de tweets con sentimiento

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

file_path = "mbsa.csv"  

# Load the latest version
df_tweets = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "gauravduttakiit/bitcoin-tweets-16m-tweets-with-sentiment-tagged",
  file_path,
)
print(df_tweets.head())


/var/folders/x7/yqjxlcpn6896ntpm60n8wg4r0000gn/T/ipykernel_52846/3501952198.py:7: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df_tweets = kagglehub.load_dataset(


         Date                                               text Sentiment
0  2019-05-27  È appena uscito un nuovo video! LES CRYPTOMONN...  Positive
1  2019-05-27  Cardano: Digitize Currencies; EOS https://t.co...  Positive
2  2019-05-27  Another Test tweet that wasn't caught in the s...  Positive
3  2019-05-27  Current Crypto Prices! \n\nBTC: $8721.99 USD\n...  Positive
4  2019-05-27  Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NO...  Positive


# Cargar de dataset de precios de bitcoin

In [ ]:
file_path = "btc_15m_data_2018_to_2025.csv"  

df_bitcoin = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "novandraanugrah/bitcoin-historical-datasets-2018-2024",
  file_path,
)

print(df_bitcoin.head())


/var/folders/x7/yqjxlcpn6896ntpm60n8wg4r0000gn/T/ipykernel_52846/2893635251.py:3: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df_bitcoin = kagglehub.load_dataset(


             Open time      Open      High       Low     Close      Volume  \
0  2018-01-01 00:00:00  13715.65  13715.65  13400.01  13556.15  123.616013   
1  2018-01-01 00:15:00  13533.75  13550.87  13402.00  13521.12   98.136430   
2  2018-01-01 00:30:00  13500.00  13545.37  13450.00  13470.41   79.904037   
3  2018-01-01 00:45:00  13494.65  13690.87  13450.00  13529.01  141.699719   
4  2018-01-01 01:00:00  13528.99  13571.74  13402.28  13445.63   72.537533   

                Close time  Quote asset volume  Number of trades  \
0  2018-01-01 00:14:59.999        1.675545e+06              1572   
1  2018-01-01 00:29:59.999        1.321757e+06              1461   
2  2018-01-01 00:44:59.999        1.078825e+06              1000   
3  2018-01-01 00:59:59.999        1.917783e+06              1195   
4  2018-01-01 01:14:59.999        9.778198e+05               898   

   Taker buy base asset volume  Taker buy quote asset volume  Ignore  
0                    63.227133                  8.5

# Leer filas del dataset de tweets y asignarle el precio segun el dataset de bitcoin

In [19]:
# Por cada fila en el dataset de bitcoin, se busca la fecha en el dataset de tweets
# y se añade la columna de sentimiento al dataset de bitcoin
df_bitcoin["Sentiment"] = None
for i in range(len(df_bitcoin)):
    date = df_bitcoin.iloc[i]["Open time"]
    # Convertir la fecha a formato datetime
    date = date.split(" ")[0]
    sentiment = df_tweets[df_tweets["Date"] == date]["Sentiment"].values
    if len(sentiment) > 0:
        df_bitcoin.at[i, "sentiment"] = sentiment[0]
    else:
        df_bitcoin.at[i, "sentiment"] = None
# Guardar el dataset de bitcoin con la columna de sentimiento
df_bitcoin.to_csv("btc_sentiment.csv", index=False)
print(df_bitcoin.head())

KeyboardInterrupt: 